In [5]:
#coloned the git repository
!git clone --recursive https://github.com/Rithik-Alias/Salary-prediction-dashboard.git
%cd /content/Salary-prediction-dashboard

Cloning into 'Salary-prediction-dashboard'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 32 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (32/32), done.
/content/Salary-prediction-dashboard


In [6]:
#import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [7]:
#imported the dataset
file = pd.read_csv('salarydata.csv')
file.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [8]:
file.loc[file['workclass'] == '?', 'workclass'] = 'Unknown'
file.loc[file['occupation'] == '?', 'occupation'] = 'Unknown'
file.loc[file['native-country'] == '?', 'native-country'] = 'Unknown'
file = file.drop(columns = ['capital-gain', 'capital-loss', 'education-num'])

In [9]:
target = file.pop('salary')
character_columns = file.dtypes.pipe(lambda x: x[x == 'object']).index
label_mapping ={}
for col in character_columns:
  file[col], label_mapping[col] = pd.factorize(file[col])
#normalizing the data
scaler = StandardScaler()  #we scale the data for larger values not affecting the smaller values
file = scaler.fit_transform(file.values)

In [10]:
#split the data
x_train, x_test, y_train, y_test = train_test_split(file,target,test_size = 0.3, random_state = 42)
#Train the model
classifier = LogisticRegression().fit(x_train, y_train)
print("---- Training complete-----")

---- Training complete-----


In [11]:
prediction = classifier.predict(x_test)
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

       <=50K       0.80      0.94      0.86      7455
        >50K       0.55      0.24      0.33      2314

    accuracy                           0.77      9769
   macro avg       0.68      0.59      0.60      9769
weighted avg       0.74      0.77      0.74      9769



In [12]:
# example of grid searching key hyperparametres for logistic regression
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
# define dataset
X, y = make_blobs(n_samples=1000, centers=2, n_features=100, cluster_std=20)
# define models and parameters
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.950333 using {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.948667 (0.021250) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.950000 (0.020166) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.948667 (0.022618) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.949000 (0.021656) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.950000 (0.020166) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.949000 (0.022561) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.948333 (0.022073) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.950333 (0.021053) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.949667 (0.021053) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.947667 (0.021084) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.948333 (0.020344) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.950000 (0.020166) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.949333 (0.018785) with: {

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [13]:
classifier = LogisticRegression(penalty = grid_result.best_params_['penalty'], C = grid_result.best_params_['C'], solver = grid_result.best_params_['solver']).fit(x_train, y_train)
prediction = classifier.predict(x_test)

print("---- Training complete-----")

---- Training complete-----


In [14]:
#generate report
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

       <=50K       0.80      0.94      0.86      7455
        >50K       0.55      0.24      0.33      2314

    accuracy                           0.77      9769
   macro avg       0.68      0.59      0.60      9769
weighted avg       0.74      0.77      0.74      9769

